In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets
path = "/content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets"

/content/drive/My Drive/Pattern Lab/Datasets/Datasets SL/classified_datasets


In [3]:
# all library imports here
import pandas as pd
import numpy as np
import os
import pickle

In [4]:
a_file = open("index_dictionary.pkl", "rb")
index_dictionary = pickle.load(a_file)
print(index_dictionary)

{'binary': [0, 163], 'CKSNAP1': [164, 195], 'CKSNAP3': [196, 259], 'CKSNAP5': [260, 355], 'CKSNAP7': [356, 483], 'CKSNAP9': [484, 643], 'ENAC5': [644, 791], 'ENAC10': [792, 939], 'DAC': [940, 981], '_EIIP': [982, 1022], 'PseEIIP': [1023, 1086], 'NCP': [1087, 1209], 'Kmer1': [1210, 1213], 'Kmer2': [1214, 1229], 'Kmer3': [1230, 1293], 'Kmer4': [1294, 1549], 'Kmer5': [1550, 2573], 'TAC': [2574, 2587]}


In [5]:
import random

def getList(dict): 
    return list(dict.keys()) 

       
keyList = getList(index_dictionary)
print(type(keyList))

random_feature = [] 

for i in range(4):
  random_feature.append(random.sample(keyList, 5))

print(random_feature )

<class 'list'>
[['Kmer4', 'ENAC10', 'DAC', '_EIIP', 'Kmer5'], ['binary', 'CKSNAP9', 'Kmer4', 'TAC', 'Kmer3'], ['PseEIIP', 'Kmer4', 'TAC', 'binary', 'Kmer3'], ['CKSNAP7', 'Kmer5', 'TAC', 'NCP', 'binary']]


In [6]:
Xa_train, Xa_test, ya_train, ya_test = np.load("xa_train.npy"), np.load("xa_test.npy"), np.load("ya_train.npy"), np.load("ya_test.npy")
Xc_train, Xc_test, yc_train, yc_test = np.load("xc_train.npy"), np.load("xc_test.npy"), np.load("yc_train.npy"), np.load("yc_test.npy")
Xd_train, Xd_test, yd_train, yd_test = np.load("xd_train.npy"), np.load("xd_test.npy"), np.load("yd_train.npy"), np.load("yd_test.npy")
print(Xa_train.shape, Xa_test.shape, ya_train.shape, ya_test.shape)
print(Xc_train.shape, Xc_test.shape, yc_train.shape, yc_test.shape)
print(Xd_train.shape, Xd_test.shape, yd_train.shape, yd_test.shape)

(3690, 2568) (262, 2568) (3690, 1) (262, 1)
(2898, 2568) (206, 2568) (2898, 1) (206, 1)
(3300, 2568) (234, 2568) (3300, 1) (234, 1)


#pa_test = np.concatenate((pa_test, dt), axis=1)
for feature_groups in random_feature:
  #feature_groups = random_feature[0] 
  no_of_features = 0
  for i in feature_groups:
    no_of_features = no_of_features + index_dictionary[i][1]-index_dictionary[i][0]+1
  print(no_of_features)

  xa_train_n, xa_test_n, ya_train_n, ya_test_n = np.zeros((Xa_train.shape[0],no_of_features)), np.zeros((Xa_test.shape[0],no_of_features)), ya_train, ya_test
  starting_index = 0
  for i in feature_groups:
    ending_index = starting_index + index_dictionary[i][1]-index_dictionary[i][0]
    print(starting_index,ending_index)
    print(index_dictionary[i][1]+1, index_dictionary[i][0])
    xa_train_n[:,starting_index:ending_index+1] = Xa_train[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    xa_test_n[:,starting_index:ending_index+1] = Xa_test[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    starting_index = ending_index+1
  print(starting_index)

In [7]:
#pa_test = np.concatenate((pa_test, dt), axis=1)
xc_trains = []
xc_tests = []
for i, feature_groups in enumerate(random_feature):
  #no_of_features = 0
  #for j in feature_groups:
   # no_of_features = no_of_features + index_dictionary[j][1]-index_dictionary[j][0]+1
  #print(no_of_features)
  xc_train_temp = np.concatenate([Xc_train[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xc_trains.append(xc_train_temp)
  xc_test_temp = np.concatenate([Xc_test[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xc_tests.append(xc_test_temp)
  print(xc_trains[i].shape)
  print(xc_tests[i].shape)

(2898, 1505)
(206, 1505)
(2898, 644)
(206, 644)
(2898, 548)
(206, 548)
(2898, 1433)
(206, 1433)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from tabulate import tabulate

In [9]:
def avg(avg_list):
  return sum(avg_list)/len(avg_list)

In [10]:
# for SVM
cv = StratifiedKFold( n_splits=5, shuffle=True)
svc = SVC() # Support vector classifier(SVC)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
ind_accuracy=[]
ind_specificity = []
ind_sensitivity = []
ind_mcc = []


svc_raws = []
headers = [ "Random-Set", "Train-Accuracy", "Test-Accuracy", "Ind-Accuracy","Train-Specificity", "Test-Specificity", "Ind-Specificity", "Train-Sensitivity", "Test-Sensitivity", "Ind-Sensitivity", "Train-MCC", "Test-MCC", "Ind-MCC"]

for i, x in enumerate(xc_trains):
  for train_index, test_index in cv.split(x, yc_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yc_train[train_index], yc_train[test_index]
      svc.fit(x_train, y_train.ravel())
      test_pred = svc.predict(x_test)
      train_pred = svc.predict(x_train)
      ind_pred = svc.predict(xc_tests[i]) # prediction of independent test dataset

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))
      ind_accuracy.append(accuracy_score(yc_test, ind_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity

      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      tn, fp, fn, tp = confusion_matrix(yc_test, ind_pred).ravel()
      ind_specificity.append(tn / (tn + fp))
      ind_sensitivity.append(tp / (tp + fn))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))
      ind_mcc.append(matthews_corrcoef(yc_test, ind_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  svc_raws.append([ "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])

print(" For SVM Classifier:\n")
table = tabulate(svc_raws, headers, tablefmt='simple')
print(table)

Random set - 1
Train-Accuracy =  0.7964122392167938
Test-Accuracy =  0.7132517420046453
Train-Specificity =  0.8083164737734669
Test-Specificity =  0.7281088175635365
Train-Sensitivity =  0.8036491690220273
Test-Sensitivity =  0.7208753964424931
Train-MCC =  0.5929990274746626
Test-MCC =  0.4273600760988794



Random set - 2
Train-Accuracy =  0.8631389195530341
Test-Accuracy =  0.7468917872669882
Train-Specificity =  0.8718084853173067
Test-Specificity =  0.7605249970170623
Train-Sensitivity =  0.8691204199379949
Test-Sensitivity =  0.7545380792524167
Train-MCC =  0.7263918089373932
Test-MCC =  0.49455471115168254



Random set - 3
Train-Accuracy =  0.8802338362749211
Test-Accuracy =  0.7532751672522978
Train-Specificity =  0.8882563744012375
Test-Specificity =  0.7685765421787375
Train-Sensitivity =  0.886106698532415
Test-Sensitivity =  0.7619554657118883
Train-MCC =  0.7605731812354677
Test-MCC =  0.5073017292389819



Random set - 4
Train-Accuracy =  0.8852659372010712
Test-Accurac

In [11]:
###  for LogisticRegression

logisticRegression = LogisticRegression(max_iter=500)
#cv = StratifiedKFold( n_splits=5, shuffle=True)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

lg_raws = []

for i, x in enumerate(xc_trains):
  for train_index, test_index in cv.split(x, yc_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yc_train[train_index], yc_train[test_index]
      logisticRegression.fit(x_train, y_train.ravel())
      test_pred = logisticRegression.predict(x_test)
      train_pred = logisticRegression.predict(x_train)
      ind_pred = logisticRegression.predict(xc_tests[i]) # prediction of independent test dataset

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))
      ind_accuracy.append(accuracy_score(yc_test, ind_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity

      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      tn, fp, fn, tp = confusion_matrix(yc_test, ind_pred).ravel()
      ind_specificity.append(tn / (tn + fp))
      ind_sensitivity.append(tp / (tp + fn))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))
      ind_mcc.append(matthews_corrcoef(yc_test, ind_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  lg_raws.append([ "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])
print(" For LogisticRegression\n")
table = tabulate(lg_raws, headers, tablefmt='orgtbl')
print(table)

Random set - 1
Train-Accuracy =  0.7817460220015396
Test-Accuracy =  0.7443100470490143
Train-Specificity =  0.7850243967748654
Test-Specificity =  0.7467223481684763
Train-Sensitivity =  0.7835757599569737
Test-Sensitivity =  0.7460502285806758
Train-MCC =  0.5635435623202517
Test-MCC =  0.48903529001194224



Random set - 2
Train-Accuracy =  0.7975759388716315
Test-Accuracy =  0.755695312965279
Train-Specificity =  0.8015012198387431
Test-Specificity =  0.7577663763274073
Train-Sensitivity =  0.7999325634022021
Test-Sensitivity =  0.7571758869348089
Train-MCC =  0.595196087748812
Test-MCC =  0.5117893062027189



Random set - 3
Train-Accuracy =  0.7972741218303537
Test-Accuracy =  0.7567315823953307
Train-Specificity =  0.8023351978023066
Test-Specificity =  0.7591528457224676
Train-Sensitivity =  0.8003257750055481
Test-Sensitivity =  0.7583110377153902
Train-MCC =  0.5946051482237019
Test-MCC =  0.5137762913928717



Random set - 4
Train-Accuracy =  0.8007031514803806
Test-Accuracy

In [12]:
###  for XGBClassifier

xgbC = XGBClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
xgbc_rows = []
for i, x in enumerate(xc_trains):
  for train_index, test_index in cv.split(x, yc_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yc_train[train_index], yc_train[test_index]
      xgbC.fit(x_train, y_train.ravel())
      test_pred = xgbC.predict(x_test)
      train_pred = xgbC.predict(x_train)
      ind_pred = xgbC.predict(xc_tests[i]) # prediction of independent test dataset

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))
      ind_accuracy.append(accuracy_score(yc_test, ind_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity


      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      tn, fp, fn, tp = confusion_matrix(yc_test, ind_pred).ravel()
      ind_specificity.append(tn / (tn + fp))
      ind_sensitivity.append(tp / (tp + fn))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))
      ind_mcc.append(matthews_corrcoef(yc_test, ind_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  xgbc_rows.append([ "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])

print("For XGBClassifier\n")
table = tabulate(xgbc_rows, headers, tablefmt='orgtbl')
print(table)

Random set -: 1
Train-Accuracy =  0.9021738491458005
Test-Accuracy =  0.7788136501697338
Train-Specificity =  0.8985513671119574
Test-Specificity =  0.7826202123851569
Train-Sensitivity =  0.8993313148007169
Test-Sensitivity =  0.7809974914158189
Train-MCC =  0.8044182394430915
Test-MCC =  0.5580126761224381



Random set -: 2
Train-Accuracy =  0.9026483961690964
Test-Accuracy =  0.7874388065034841
Train-Specificity =  0.897687587396983
Test-Specificity =  0.7870910392554589
Train-Sensitivity =  0.8987316286171456
Test-Sensitivity =  0.7873277899611345
Train-MCC =  0.8053755760331613
Test-MCC =  0.5752494759525802



Random set -: 3
Train-Accuracy =  0.8994425016584683
Test-Accuracy =  0.7931931788855141
Train-Specificity =  0.8955600844961469
Test-Specificity =  0.7950260509883468
Train-Sensitivity =  0.8963814366658677
Test-Sensitivity =  0.7944759525203416
Train-MCC =  0.7989453457635528
Test-MCC =  0.5867526398291939



Random set -: 4
Train-Accuracy =  0.9007938044908679
Test-Accu

In [13]:
###  for AdaBoostClassifier

adaBC = AdaBoostClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
adabc_rows = []

for i, x in enumerate(xc_trains):
  for train_index, test_index in cv.split(x, yc_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yc_train[train_index], yc_train[test_index]
      adaBC.fit(x_train, y_train.ravel())
      test_pred = adaBC.predict(x_test)
      train_pred = adaBC.predict(x_train)
      ind_pred = adaBC.predict(xc_tests[i]) # prediction of independent test dataset

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))
      ind_accuracy.append(accuracy_score(yc_test, ind_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity


      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      tn, fp, fn, tp = confusion_matrix(yc_test, ind_pred).ravel()
      ind_specificity.append(tn / (tn + fp))
      ind_sensitivity.append(tp / (tp + fn))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))
      ind_mcc.append(matthews_corrcoef(yc_test, ind_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  adabc_rows.append([ "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])

print("For AdaBoostClassifier\n")
table = tabulate(adabc_rows, headers, tablefmt='orgtbl')
print(table)

Random set -: 1
Train-Accuracy =  0.8026224076085278
Test-Accuracy =  0.7432779465189684
Train-Specificity =  0.8015886168218737
Test-Specificity =  0.7329340174203556
Train-Sensitivity =  0.8020537613434747
Test-Sensitivity =  0.7386285634373702
Train-MCC =  0.6053015884720152
Test-MCC =  0.4872278720104329



Random set -: 2
Train-Accuracy =  0.8026657900875872
Test-Accuracy =  0.7475832291108332
Train-Specificity =  0.8030540596828419
Test-Specificity =  0.736713995943205
Train-Sensitivity =  0.8029383406325561
Test-Sensitivity =  0.7424939115561112
Train-MCC =  0.6053845348221236
Test-MCC =  0.49582617097183



Random set -: 3
Train-Accuracy =  0.8004948430287219
Test-Accuracy =  0.7510926488396561
Train-Specificity =  0.8002066287822439
Test-Specificity =  0.7425780535337868
Train-Sensitivity =  0.8003698353135591
Test-Sensitivity =  0.7471008493781547
Train-MCC =  0.6010404003975724
Test-MCC =  0.5026948462248411



Random set -: 4
Train-Accuracy =  0.8017168913737697
Test-Accura

In [14]:
###  for RandomForestClassifier

rfC = RandomForestClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
rfc_rows = []

for i, x in enumerate(xc_trains):
  for train_index, test_index in cv.split(x, yc_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yc_train[train_index], yc_train[test_index]
      rfC.fit(x_train, y_train.ravel())
      test_pred = rfC.predict(x_test)
      train_pred = rfC.predict(x_train)
      ind_pred = rfC.predict(xc_tests[i]) # prediction of independent test dataset

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))
      ind_accuracy.append(accuracy_score(yc_test, ind_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity


      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      tn, fp, fn, tp = confusion_matrix(yc_test, ind_pred).ravel()
      ind_specificity.append(tn / (tn + fp))
      ind_sensitivity.append(tp / (tp + fn))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))
      ind_mcc.append(matthews_corrcoef(ya_test, ind_pred))


  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  rfc_rows.append([ "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])

print("For RandomForestClassifier\n")
table = tabulate(rfc_rows, headers, tablefmt='orgtbl')
print(table)

ValueError: ignored